In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

import pandas as pd
import numpy as np
import os
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as torch_optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from datetime import datetime

In [2]:
os.getcwd()

'/Users/korniltsevdmitry/Desktop/python/projects/recommendation_systems/notebooks'

In [3]:
os.chdir('../')

In [4]:
from core.alternative_emb_cat import TabularDataset, FeedForwardNN

In [5]:
data = pd.read_csv('./data/result_df.csv', nrows=2000)

In [6]:
data.head(2)

,hash_inn,okved2,region,0_count_kt,1_count_kt,10_count_kt,1000_count_kt,11_count_kt,12_count_kt,13_count_kt,...,share_72_dt_dt,share_73_dt_dt,share_74_dt_dt,share_75_dt_dt,share_76_dt_dt,share_77_dt_dt,share_78_dt_dt,share_79_dt_dt,share_8_dt_dt,share_9_dt_dt
0,61058,34,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8311,18,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
cat_col = ["region"]
cont_col = [col for col in data.columns if 'share' in col]
target_col = ["okved2"]
output_feature = "okved2"

In [8]:
data = data[cont_col + cat_col + target_col]
data.fillna(0, inplace=True)

In [9]:
len(data.okved2.unique())

75

In [12]:
label_encoders = {}
for col in cat_col:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col]) 
    data[col] = data[col].astype('category')

In [13]:
Y = LabelEncoder().fit_transform(data[target_col])

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
X = data[cont_col + cat_col]

In [15]:
# emb_cols_dict = {n: len(col.cat.categories) for n, col in X[cat_col].items() if len(col.cat.categories) > 2}
# emb_cols = emb_cols_dict.keys()
# emb_sizes = [(c, min(50, (c+1)//2)) for _,c in emb_cols_dict.items()]

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)
X_train.head(2)

,share_0_kt,share_1_kt,share_10_kt,share_1000_kt,share_11_kt,share_12_kt,share_13_kt,share_14_kt,share_15_kt,share_16_kt,...,share_73_dt_dt,share_74_dt_dt,share_75_dt_dt,share_76_dt_dt,share_77_dt_dt,share_78_dt_dt,share_79_dt_dt,share_8_dt_dt,share_9_dt_dt,region
81,0.0,0.0,0.0,0.285714,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71
915,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71


In [36]:
len(data.okved2.unique())

75

In [17]:
cat_dims = [int(data[col].nunique()) for col in cat_col]
emb_dims = [(x, min(50, (x + 1) // 2)) for x in cat_dims]

In [18]:
dataset = TabularDataset(data=data, cat_cols=cat_col,
                             output_col=output_feature)

In [39]:
batchsize = 128
dataloader = DataLoader(dataset, batchsize, shuffle=True, num_workers=1)

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeedForwardNN(emb_dims, no_of_cont=len(cont_col), lin_layer_sizes=[250, 500],
                          output_size=len(data.okved2.unique()), emb_dropout=0.04,
                          lin_layer_dropouts=[0.001,0.01]).to(device)

In [42]:
no_of_epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
for epoch in range(no_of_epochs):
    print("epoch: {}".format(epoch))
    for y, cont_x, cat_x in dataloader:
        cat_x = cat_x.to(device)
        cont_x = cont_x.to(device)
        y  = y.to(device)
        

        # Forward Pass
        preds = model(cont_x, cat_x)
#         y = y.squeeze()
        loss = criterion(preds, y)

        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

epoch: 0


RuntimeError: 1D target tensor expected, multi-target not supported